# Flash Evaluation on Streamspot Dataset:

This notebook is dedicated to evaluating Flash on the Streamspot dataset, which are graph-level in nature. We employ Flash in graph-level detection mode to analyze this dataset effectively. Upon completion of the notebook execution, the results will be presented.

## Dataset Access: 
- The Streamspot dataset can be accessed at the following link: [Streamspot Dataset](https://github.com/sbustreamspot/sbustreamspot-data).
- Please download the required data files from the provided link.

## Data Parsing and Execution:
- Utilize the parser included in this notebook to process the downloaded files.
- To obtain the evaluation results, execute all cells within this notebook.

## Model Training and Execution Flexibility:
- By default, the notebook operates using pre-trained model weights.
- Additionally, this notebook offers the flexibility to set parameters for training Graph Neural Networks (GNNs) and word2vec models from scratch.
- You can then utilize these freshly trained models to conduct the evaluation. 

Follow these guidelines for a thorough and efficient analysis of the Streamspot dataset using Flash.

In [61]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import os
import torch.nn.functional as F
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import multiprocessing

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline
print(device)

cuda


In [62]:
Train_Gnn = False
Train_Word2vec = False

In [63]:
from pprint import pprint
import gzip
from sklearn.manifold import TSNE
import json
import copy
import os
import tqdm

In [64]:
import os.path as osp
import csv
def show(str):
	print (str + ' ' + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time())))

def parse_data():
    os.system('tar -zxvf all.tar.gz')

    show('Start processing.')
    data = []
    gId = -1
    with open('all.tsv') as f:
        tsvreader = csv.reader(f, delimiter='\t')
        for row in tsvreader:
            if int(row[5]) != gId:
                gId = int(row[5])
                show('Graph ' + str(gId))
                scene = int(gId/100)+1
                if not osp.exists('streamspot/'+str(scene)):
                    os.system('mkdir streamspot/'+str(scene))
                ff = open('streamspot/'+str(scene)+'/'+str(gId)+'.txt', 'w')
            ff.write(str(row[0])+'\t'+str(row[1])+'\t'+str(row[2])+'\t'+str(row[3])+'\t'+str(row[4])+'\t'+str(row[5])+'\n')
    os.system('rm all.tsv')
    show('Done.')

In [65]:
# parse_data()

In [66]:
def prepare_graph(df):
    nodes, labels, edges = {}, {}, []
    dummies = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}

    for _, row in df.iterrows():
        actor_id, object_id = row["actorID"], row["objectID"]
        action = row["action"]

        for entity_id in [actor_id, object_id]:
            nodes.setdefault(entity_id, []).append(action)
            if entity_id == actor_id:
                labels[entity_id] = dummies[row['actor_type']]
            else:
                labels[entity_id] = dummies[row['object']]

        edges.append((actor_id, object_id))

    features, feat_labels, edge_index, mapping = [], [], [[], []], []
    index_map = {}

    for key, value in nodes.items():
        index_map[key] = len(features)
        features.append(value)
        feat_labels.append(labels[key])
        mapping.append(key)

    for source, target in edges:
        edge_index[0].append(index_map[source])
        edge_index[1].append(index_map[target])

    return features, feat_labels, edge_index, mapping

In [67]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, GATConv
import torch.nn as nn


class GCN(torch.nn.Module):
    def __init__(self,in_channel,out_channel):
        super().__init__()
        self.conv1 = SAGEConv(in_channel, 32, normalize=True)
        self.conv2 = SAGEConv(32, out_channel, normalize=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        return x

In [68]:
def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

In [69]:
from gensim.models.callbacks import CallbackAny2Vec
import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time

class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('trained_weights/streamspot/streamspot.model')
        self.epoch += 1

In [70]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [71]:
logger = EpochLogger()
saver = EpochSaver()

In [72]:
if Train_Word2vec:
    phrases = []
    for i in tqdm(range(50)):
        print(i)
        f = open(f"streamspot/{i}.txt")
        data = f.read().split('\n')

        data = [line.split('\t') for line in data]
        df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
        df = df.dropna()
        docs,labels,edges,mapp = prepare_graph(df)
        phrases = phrases + docs
        
    word2vec = Word2Vec(sentences=phrases, vector_size=30, window=10, min_count=1, workers=8,epochs=100,callbacks=[saver,logger])

In [73]:
from sklearn.utils import class_weight
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GCN(30,8).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [74]:
import math
import torch
import numpy as np
from gensim.models import Word2Vec

class PositionalEncoder:

    def __init__(self, d_model, max_len=100000):
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(max_len, d_model)
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)

    def embed(self, x):
        return x + self.pe[:x.size(0)]

def infer(document):
    word_embeddings = [w2vmodel.wv[word] for word in document if word in  w2vmodel.wv]
    
    if not word_embeddings:
        return np.zeros(20)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float)
    if len(document) < 100000:
        output_embedding = encoder.embed(output_embedding)

    output_embedding = output_embedding.detach().cpu().numpy()
    return np.mean(output_embedding, axis=0)

encoder = PositionalEncoder(30)
w2vmodel = Word2Vec.load("trained_weights/streamspot/streamspot.model")

In [75]:
from torch_geometric import utils

if Train_Gnn:
    for i in tqdm(range(300)):
        f = open(f"streamspot/{i}.txt")
        data = f.read().split('\n')

        data = [line.split('\t') for line in data]
        df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
        df = df.dropna()
        phrases,labels,edges,mapp = prepare_graph(df)

        criterion = CrossEntropyLoss()

        nodes = [infer(x) for x in phrases]
        nodes = np.array(nodes)  

        graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))

        model.train()
        optimizer.zero_grad() 
        out = model(graph.x, graph.edge_index) 
        loss = criterion(out, graph.y) 
        loss.backward() 
        optimizer.step() 

        _ , indices = out.sort(dim=1,descending=True)
        pred = indices[:,0]
        cond = pred == graph.y

        print(cond.sum() / len(graph.y))

        torch.save(model.state_dict(), f'trained_weights/streamspot/lstreamspot.pth')

### Validation

In [76]:
model.load_state_dict(torch.load(f'trained_weights/streamspot/lstreamspot.pth', map_location=torch.device('cpu')))
model.eval()

for i in tqdm(range(400,450)):
    f = open(f"streamspot/5/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df = df.dropna()
    
    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)
    
    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)

    out = model(graph.x, graph.edge_index)

    sorted, indices = out.sort(dim=1,descending=True)
    conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
    conf = (conf - conf.min()) / conf.max()

    pred = indices[:,0]
    cond = ~(pred == graph.y)
    
    print(cond.sum().item(), (cond.sum().item() / len(cond))*100)

  2%|█▉                                                                                            | 1/50 [00:21<17:15, 21.12s/it]

106 1.214342994615649


  4%|███▊                                                                                          | 2/50 [00:41<16:37, 20.79s/it]

110 1.247731397459165


  6%|█████▋                                                                                        | 3/50 [00:55<13:45, 17.57s/it]

...

109 1.2227956024231545


 98%|███████████████████████████████████████████████████████████████████████████████████████████▏ | 49/50 [14:38<00:15, 15.76s/it]

94 1.0604693140794224


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [14:51<00:00, 17.84s/it]

97 1.1079383209594518


### Testing

In [82]:
thresh = 200
correct_benign = 0
correct_attack = 0

In [83]:
model.load_state_dict(torch.load(f'trained_weights/streamspot/lstreamspot.pth',map_location=torch.device('cpu')))
model.eval()

GCN(
  (conv1): SAGEConv(30, 32, aggr=mean)
  (conv2): SAGEConv(32, 8, aggr=mean)
)

In [84]:
for i in tqdm(range(450,600)):
    scene=int(i/100)+1
    f = open(f"streamspot/{scene}/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df = df.dropna()
    
    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)
    
    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)

    out = model(graph.x, graph.edge_index)

    sorted, indices = out.sort(dim=1,descending=True)
    conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
    conf = (conf - conf.min()) / conf.max()

    pred = indices[:,0]
    cond = ~(pred == graph.y)

    if cond.sum() <= thresh:
         correct_benign = correct_benign + 1
    
    print(cond.sum().item(), (cond.sum().item() / len(cond))*100)

  1%|▌                                                                                            | 1/150 [00:16<42:03, 16.94s/it]

113 1.2537445911461222


  1%|█▏                                                                                           | 2/150 [00:27<33:14, 13.48s/it]

89 1.0111338332197228


  2%|█▊                                                                                           | 3/150 [00:42<34:22, 14.03s/it]

...

84 0.9413874257536703


 99%|██████████████████████████████████████████████████████████████████████████████████████████▍| 149/150 [37:57<00:14, 14.56s/it]

79 0.8767062479192098


100%|███████████████████████████████████████████████████████████████████████████████████████████| 150/150 [38:10<00:00, 15.27s/it]

80 0.8917623453349682


In [85]:
for i in tqdm(range(300,400)):
    scene=int(i/100)+1
    f = open(f"streamspot/{scene}/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df = df.dropna()
    
    phrases,labels,edges,mapp = prepare_graph(df)
  
    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)
    
    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)

    out = model(graph.x, graph.edge_index)

    sorted, indices = out.sort(dim=1,descending=True)
    conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
    conf = (conf - conf.min()) / conf.max()

    pred = indices[:,0]
    cond = ~(pred == graph.y)

    if cond.sum() > thresh:
         correct_attack = correct_attack + 1
            
    print(cond.sum().item(), (cond.sum().item() / len(cond))*100)

  1%|▉                                                                                            | 1/100 [00:01<02:56,  1.78s/it]

1072 11.995076647644623


  2%|█▊                                                                                           | 2/100 [00:03<03:06,  1.91s/it]

1076 12.031756681203177


  3%|██▊                                                                                          | 3/100 [00:05<02:58,  1.84s/it]

...

1075 12.01923076923077


 99%|███████████████████████████████████████████████████████████████████████████████████████████ | 99/100 [03:02<00:01,  1.93s/it]

1075 12.020574751202059


100%|███████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:04<00:00,  1.85s/it]

1075 12.025953686094642


In [86]:
TOTAL_ATTACKS = 100
TOTAL_BENIGN = 150

def calculate_metrics(correct_attack, correct_benign):
    TP = correct_attack
    FP = TOTAL_BENIGN - correct_benign
    TN = correct_benign
    FN = TOTAL_ATTACKS - correct_attack

    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0

    print(f"Number of True Positives: {TP}")
    print(f"Number of False Positives: {FP}")
    print(f"Number of False Negatives: {FN}")
    print(f"Number of True Negatives: {TN}\n")

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")

    fscore = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f"Fscore: {fscore}\n")

calculate_metrics(correct_attack, correct_benign)

Number of True Positives: 95
Number of False Positives: 0
Number of False Negatives: 5
Number of True Negatives: 150

Precision: 1.0
Recall: 0.95
Fscore: 0.9743589743589743

